In [ ]:
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy.random import seed

In [ ]:
#import 

Data_set= pd.read_csv("/home/effrancodelos/Deep_learnig/Data_Nova/COAD/COAD_COX/data_Merge_Colon_t.csv")
Data_set.head(n=20)



In [ ]:
#Create a data-set with the string columns

Pacients = Data_set['Pacients']
#Radiation = Data_set['radiations']


In [ ]:
Data_set.drop(['Pacients'], axis=1, inplace=True)
#Data_set.drop(Data_set.columns[0], axis=1, inplace=True)

In [ ]:
print (Data_set.shape)

In [ ]:
#data tranforamtion

Data_set_scaled =  minmax_scale(Data_set, axis = 0)
#Columns inputs number
ncol = Data_set_scaled.shape[1]



In [ ]:
#Split the dataset 
X_train, X_test = train_test_split(Data_set_scaled, train_size = 0.8, random_state = seed(2017))


In [ ]:
#add noisy to the main data_set
noise_factor = 0.5
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=0.3, size=X_train.shape)

In [ ]:
#add noisy to the main data_set
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=0.3, size=X_test.shape)

In [ ]:
#Clip (limit) the values in an array
#X_train_noisy = np.clip(X_train_noisy, 0., 1.)
#X_test_noisy= np.clip(X_train_noisy, 0., 1.)

In [ ]:
#encoding
encoding_dim = 100

In [ ]:
input_dim = Input(shape = (ncol, ))
#Encoder
encoded1 = Dense(units=500, activation='tanh')(input_dim)
encoded2 = Dense(encoding_dim, activation='tanh')(encoded1)



#decoded 
decoded1 = Dense(500, activation='tanh')(encoded2)
decoded2= Dense(ncol, activation='sigmoid')(decoded1)
# Building autoencoder


In [ ]:
# Building autoencoder
autoencoder=Model(input_dim, decoded2)

In [ ]:
#extracting encoder
encoder = Model(input_dim, encoded2)

In [ ]:
# compiling the autoencoder
autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')# BEST
#autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
#fit the network
autoencoder.fit(X_train_noisy, X_train_noisy,
                epochs=40,
                batch_size=32,
                shuffle=True,
                validation_data=(X_test, X_test))

In [ ]:
encoder = Model(inputs = input_dim, outputs = encoded2)
encoded_input = Input(shape = (encoding_dim, ))

In [ ]:
Data_set_scaled_noisy= Data_set_scaled + noise_factor * np.random.normal(loc=0.0, scale=0.3, size=Data_set_scaled.shape)

In [ ]:
encoded_train = pd.DataFrame(encoder.predict(Data_set_scaled_noisy))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
encoded_train['Pacients'] = Pacients
#encoded_train['Radiations'] = Radiation

In [ ]:
print(encoded_train.shape)
encoded_train.head()

In [ ]:
encoded_train.to_csv("/home/effrancodelos/Deep_learnig/Data_Nova/COAD/COAD_COX/data_Merge_Colon_Cox_Denoising_1.csv", index=False)